#### **Lavish Gulati - 170101082**
# **CS565 - Intelligent Systems and Interfaces - Assignment 1**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install indic-nlp-library

In [ ]:
from nltk.probability import FreqDist

### 1.3.1 - Analysis using existing NLP tools

In [ ]:
import codecs
import nltk
from nltk.util import ngrams
import numpy as np
from matplotlib import pyplot
import math
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
eng_text = codecs.open('/content/drive/My Drive/CS565/Assignment 1/en_wiki.txt', 'r').read()
hin_text = codecs.open('/content/drive/My Drive/CS565/Assignment 1/hi_wiki.txt', 'r').read()
small_ratio = 0.1
eng_small_text = eng_text[0:int(small_ratio*len(eng_text))]
hin_small_text = hin_text[0:int(small_ratio*len(hin_text))]
print('% of text taken:', (len(hin_small_text)/len(hin_text))*100)

% of text taken: 9.999999753385895


In [ ]:
def extract_ngrams(tok_list, num):
  n_grams = ngrams(tok_list, num)
  return [' '.join(grams) for grams in n_grams]

def plot_freq_dist(freq_dist, n, lang="English", num_samples=100):
    frequencies = [freq_dist[sample] for sample,_ in freq_dist.most_common(num_samples)]
    pos = np.arange(num_samples)
    width = 1.0
    
    ngram_label = lang
    if n == 1:
      ngram_label += " unigram"
    elif n == 2:
      ngram_label += " bigram"
    elif n == 3:
      ngram_label += " trigram"

    ax = pyplot.axes()
    ax.set_xticks(pos + (width / 2))
    ax.set_xticklabels(pos)
    ax.set_xlabel(ngram_label + " rank")
    ax.set_ylabel(ngram_label + " frequency")
    ax.set_title('Frequency vs rank (' + ngram_label + ')')
    ax.grid(True)
    pyplot.ylim(0, math.ceil(max(frequencies)/5)*5)
    pyplot.bar(pos, frequencies, width, color='b', edgecolor='k')
    figure = pyplot.gcf()
    figure.savefig(ngram_label, dpi=figure.dpi)
    pyplot.close()

#### 1 - Sentence Segmentation and Word Tokenization

In [ ]:
!pip install stanza

     |████████████████████████████████| 235kB 8.3MB/s 


In [ ]:
import stanza
stanza.download('hi')

2020-09-29 13:06:41 INFO: Downloading default packages for language: hi (Hindi)...
2020-09-29 13:07:04 INFO: Finished downloading models and saved to /root/stanza_resources.


##### Sentence Segmentation - English

In [ ]:
eng_sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
eng_sent_tokens = eng_sent_tokenizer.tokenize(eng_text)

In [ ]:
print('Number of sentence tokens:', len(eng_sent_tokens))
print('Sample output:', eng_sent_tokens[0:5])

Number of sentence tokens: 761582
Sample output: ['The word "atom" was coined by ancient Greek philosophers.', 'However, these ideas were founded in philosophical and theological reasoning rather than evidence and experimentation.', 'As a result, their views on what atoms look like and how they behave were incorrect.', 'They also could not convince everybody, so atomism was but one of a number of competing theories on the nature of matter.', 'It was not until the 19th century that the idea was embraced and refined by scientists, when the blossoming science of chemistry produced discoveries that only the concept of atoms could explain.']


In [ ]:
eng_sent_tokens = nltk.tokenize.regexp_tokenize(eng_text, pattern='(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', gaps=True)

In [ ]:
print('Number of sentence tokens:', len(eng_sent_tokens))
print('Sample output:', eng_sent_tokens[0:5])

Number of sentence tokens: 750330
Sample output: ['The word "atom" was coined by ancient Greek philosophers.', 'However, these ideas were founded in philosophical and theological reasoning rather than evidence and experimentation.', 'As a result, their views on what atoms look like and how they behave were incorrect.', 'They also could not convince everybody, so atomism was but one of a number of competing theories on the nature of matter.', 'It was not until the 19th century that the idea was embraced and refined by scientists, when the blossoming science of chemistry produced discoveries that only the concept of atoms could explain.']


##### Sentence Segmentation - Hindi

In [ ]:
from indicnlp.tokenize import sentence_tokenize

In [ ]:
hin_sent_tokens = sentence_tokenize.sentence_split(hin_text, lang='hi')

In [ ]:
print('Number of sentence tokens:', len(hin_sent_tokens))
print('Sample output:', hin_sent_tokens[0:3])

Number of sentence tokens: 348593
Sample output: ['मास्टर ऑफ़ हेल्थ एडमिनिस्ट्रेशन या मास्टर ऑफ हेल्थकेयर एडमिनिस्ट्रेशन (एमएचए या एम. एच. ए) स्नातकोत्तर (पोस्ट ग्रेजुएशन) की एक पेशेवर डिग्री है जो स्वास्थ्य प्रशासन के क्षेत्र में दी जाती हैं।', 'यह उन छात्रों को प्रदान की जाती हैं जिन्होंने स्वास्थ्य प्रशासन, अस्पताल प्रबंधन एवं अन्य स्वास्थ्य सेवा संगठनों के क्षेत्र में जरूरी ज्ञान और दक्षता हासिल की हैं।', 'इन पाठ्यक्रमो में परिस्थितियों के अनुसार इनके सरंचना में अंतर हो सकता हैं हालांकि व्यवसायी-शिक्षक मॉडल कार्यक्रम आमतौर पर चिकित्सा, स्वास्थ्य व्यवसायों या संबद्ध स्वास्थ्य के कॉलेजों में पाए जाते हैं, कक्षा-आधारित कार्यक्रम व्यवसाय या सार्वजनिक स्वास्थ्य के कॉलेजों में होते हैं।']


In [ ]:
hin_sent_tokenizer = stanza.Pipeline(lang='hi', processors='tokenize')
hin_sent_tokens = hin_sent_tokenizer(hin_small_text).sentences
hin_sent_tokens = [sentence.text for sentence in hin_sent_tokens]

2020-09-29 08:05:48 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |

2020-09-29 08:05:48 INFO: Use device: cpu
2020-09-29 08:05:48 INFO: Loading: tokenize
2020-09-29 08:05:48 INFO: Done loading processors!


In [ ]:
print('Number of sentence tokens:', len(hin_sent_tokens))
print('Sample output:', hin_sent_tokens[0:3])

Number of sentence tokens: 36436
Sample output: ['मास्टर ऑफ़ हेल्थ एडमिनिस्ट्रेशन या मास्टर ऑफ हेल्थकेयर एडमिनिस्ट्रेशन (एमएचए या एम.एच.ए) स्नातकोत्तर (पोस्ट ग्रेजुएशन) की एक पेशेवर डिग्री है जो स्वास्थ्य प्रशासन के क्षेत्र में दी जाती हैं।', 'यह उन छात्रों को प्रदान की जाती हैं जिन्होंने स्वास्थ्य प्रशासन, अस्पताल प्रबंधन एवं अन्य स्वास्थ्य सेवा संगठनों के क्षेत्र में जरूरी ज्ञान और दक्षता हासिल की हैं।', 'इन पाठ्यक्रमो में परिस्थितियों के अनुसार इनके सरंचना में अंतर हो सकता हैं']


##### Word Tokenization - English

In [ ]:
from nltk.tokenize import TreebankWordTokenizer, WordPunctTokenizer

In [ ]:
eng_word_tokenizer = TreebankWordTokenizer() 
eng_word_tokens = eng_word_tokenizer.tokenize(eng_small_text)
print('% of text taken:', (len(hin_small_text)/len(hin_text))*100)

% of text taken: 9.999999753385895


In [ ]:
print('Number of word tokens:', len(eng_word_tokens))
print('Sample output:', eng_word_tokens[0:20])

Number of word tokens: 1974621
Sample output: ['The', 'word', '``', 'atom', "''", 'was', 'coined', 'by', 'ancient', 'Greek', 'philosophers.', 'However', ',', 'these', 'ideas', 'were', 'founded', 'in', 'philosophical', 'and']


In [ ]:
eng_word_tokenizer = WordPunctTokenizer() 
eng_word_tokens = eng_word_tokenizer.tokenize(eng_text)

In [ ]:
print('Number of word tokens:', len(eng_word_tokens))
print('Sample output:', eng_word_tokens[0:20])

Number of word tokens: 20372526
Sample output: ['The', 'word', '"', 'atom', '"', 'was', 'coined', 'by', 'ancient', 'Greek', 'philosophers', '.', 'However', ',', 'these', 'ideas', 'were', 'founded', 'in', 'philosophical']


##### Word Tokenization - Hindi

In [68]:
from indicnlp.tokenize import indic_tokenize

In [69]:
hin_word_tokens = indic_tokenize.trivial_tokenize(hin_text)

In [70]:
print('Number of word tokens:', len(hin_word_tokens))
print('Sample Output:', hin_word_tokens[0:20])

Number of word tokens: 8640033
Sample Output: ['मास्टर', 'ऑफ़', 'हेल्थ', 'एडमिनिस्ट्रेशन', 'या', 'मास्टर', 'ऑफ', 'हेल्थकेयर', 'एडमिनिस्ट्रेशन', '(', 'एमएचए', 'या', 'एम', '.', 'एच', '.', 'ए', ')', 'स्नातकोत्तर', '(']


In [ ]:
hin_word_tokenizer = stanza.Pipeline(lang='hi', processors='tokenize', tokenize_no_ssplit=True)
hin_sent_tokens = hin_word_tokenizer(hin_small_text).sentences
hin_word_tokens = []
for i, sentence in enumerate(hin_sent_tokens):
  for token in sentence.tokens:
    hin_word_tokens.append(token.text)

2020-09-29 08:40:58 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |

2020-09-29 08:40:58 INFO: Use device: cpu
2020-09-29 08:40:58 INFO: Loading: tokenize
2020-09-29 08:40:58 INFO: Done loading processors!


In [ ]:
print('Number of word tokens:', len(hin_word_tokens))
print('Sample Output:', hin_word_tokens[0:20])

Number of word tokens: 840150
Sample Output: ['मास्टर', 'ऑफ़', 'हेल्थ', 'एडमिनिस्ट्रेशन', 'या', 'मास्टर', 'ऑफ', 'हेल्थकेयर', 'एडमिनिस्ट्रेशन', '(', 'एमएचए', 'या', 'एम.एच.ए', ')', 'स्नातकोत्तर', '(', 'पोस्ट', 'ग्रेजुएशन', ')', 'की']


#### 2 - Unigrams
Dependencies: eng_word_tokens, hin_word_tokens

##### Unigrams - English

In [ ]:
eng_unigrams = extract_ngrams(eng_word_tokens, 1)
print('Number of word tokens as input:', len(eng_word_tokens))

Number of word tokens as input: 20372526


In [ ]:
print('Number of unigrams obtained:', len(eng_unigrams))
print('Sample output', eng_unigrams[0:20])

Number of unigrams obtained: 20372526
Sample output ['The', 'word', '"', 'atom', '"', 'was', 'coined', 'by', 'ancient', 'Greek', 'philosophers', '.', 'However', ',', 'these', 'ideas', 'were', 'founded', 'in', 'philosophical']


In [ ]:
eng_uni_freq_dist = nltk.FreqDist(eng_unigrams)
plot_freq_dist(eng_uni_freq_dist, 1, "English")
print('Type of unigrams:', len(eng_uni_freq_dist))
print('Sample Output:', eng_uni_freq_dist.most_common(10))

Type of unigrams: 286686
Sample Output: [('the', 1084635), (',', 994858), ('.', 862133), ('of', 646923), ('and', 499999), ('in', 379124), ('to', 356109), ('a', 334452), ('"', 308618), ('was', 210438)]


##### Unigrams - Hindi

In [ ]:
hin_unigrams = extract_ngrams(hin_word_tokens, 1)
print('Number of word tokens as input:', len(hin_word_tokens))

Number of word tokens as input: 8640033


In [ ]:
print('Number of unigrams obtained:', len(hin_unigrams))
print('Sample output', hin_unigrams[0:20])

Number of unigrams obtained: 8640033
Sample output ['मास्टर', 'ऑफ़', 'हेल्थ', 'एडमिनिस्ट्रेशन', 'या', 'मास्टर', 'ऑफ', 'हेल्थकेयर', 'एडमिनिस्ट्रेशन', '(', 'एमएचए', 'या', 'एम', '.', 'एच', '.', 'ए', ')', 'स्नातकोत्तर', '(']


In [ ]:
hin_uni_freq_dist = nltk.FreqDist(hin_unigrams)
plot_freq_dist(hin_uni_freq_dist, 1, "Hindi")
print('Type of unigrams:', len(hin_uni_freq_dist))
print('Sample Output:', hin_uni_freq_dist.most_common(10))

Type of unigrams: 322350
Sample Output: [('के', 357833), ('।', 321526), ('में', 268249), (',', 267743), ('है', 232156), ('की', 171779), ('और', 155743), ('से', 134901), ('का', 122948), ('को', 115876)]


#### 3 - Bigrams
Dependencies: eng_word_tokens, hin_word_tokens

##### Bigrams - English

In [ ]:
eng_bigrams = extract_ngrams(eng_word_tokens, 2)
print('Number of word tokens as input:', len(eng_word_tokens))

Number of word tokens as input: 20372526


In [ ]:
print('Number of bigrams obtained:', len(eng_bigrams))
print('Sample output', eng_bigrams[0:10])

Number of bigrams obtained: 20372525
Sample output ['The word', 'word "', '" atom', 'atom "', '" was', 'was coined', 'coined by', 'by ancient', 'ancient Greek', 'Greek philosophers']


In [ ]:
eng_bi_freq_dist = nltk.FreqDist(eng_bigrams)
plot_freq_dist(eng_bi_freq_dist, 2, "English")
print('Type of bigrams:', len(eng_bi_freq_dist))
print('Sample Output:', eng_bi_freq_dist.most_common(10))

Type of bigrams: 3977129
Sample Output: [('of the', 179431), ('. The', 140543), (', and', 114438), ("' s", 105543), ('in the', 104836), (', the', 73479), ('to the', 58240), ('. In', 56002), ('and the', 47484), ('on the', 32972)]


##### Bigrams - Hindi

In [ ]:
hin_bigrams = extract_ngrams(hin_word_tokens, 2)
print('Number of word tokens as input:', len(hin_word_tokens))

Number of word tokens as input: 8640033


In [ ]:
print('Number of bigrams obtained:', len(hin_bigrams))
print('Sample output', hin_bigrams[0:10])

Number of bigrams obtained: 8640032
Sample output ['मास्टर ऑफ़', 'ऑफ़ हेल्थ', 'हेल्थ एडमिनिस्ट्रेशन', 'एडमिनिस्ट्रेशन या', 'या मास्टर', 'मास्टर ऑफ', 'ऑफ हेल्थकेयर', 'हेल्थकेयर एडमिनिस्ट्रेशन', 'एडमिनिस्ट्रेशन (', '( एमएचए']


In [ ]:
hin_bi_freq_dist = nltk.FreqDist(hin_bigrams)
plot_freq_dist(hin_bi_freq_dist, 2, "Hindi")
print('Type of bigrams:', len(hin_bi_freq_dist))
print('Sample Output:', hin_bi_freq_dist.most_common(10))

Type of bigrams: 2392979
Sample Output: [('है ।', 125369), ('के लिए', 49250), ('हैं ।', 48556), ('है ,', 32259), ('जाता है', 25250), ('था ।', 24258), ('के साथ', 20999), ('रूप में', 18031), ('के रूप', 16812), ('है कि', 15652)]


#### 4 - Trigrams
Dependencies: eng_word_tokens, hin_word_tokens

##### Trigrams - English

In [ ]:
eng_trigrams = extract_ngrams(eng_word_tokens, 3)
print('Number of word tokens as input:', len(eng_word_tokens))

Number of word tokens as input: 20372526


In [ ]:
print('Number of trigrams obtained:', len(eng_trigrams))
print('Sample output', eng_trigrams[0:10])

Number of trigrams obtained: 20372524
Sample output ['The word "', 'word " atom', '" atom "', 'atom " was', '" was coined', 'was coined by', 'coined by ancient', 'by ancient Greek', 'ancient Greek philosophers', 'Greek philosophers .']


In [ ]:
eng_tri_freq_dist = nltk.FreqDist(eng_trigrams)
plot_freq_dist(eng_tri_freq_dist, 3, "English")
print('Type of trigrams:', len(eng_tri_freq_dist))
print('Sample Output:', eng_tri_freq_dist.most_common(5))

Type of trigrams: 10878682
Sample Output: [(', and the', 14397), ('. In the', 13244), ('the age of', 11064), ('under the age', 9928), ('the United States', 9467)]


##### Trigrams - Hindi

In [ ]:
hin_trigrams = extract_ngrams(hin_word_tokens, 3)
print('Number of word tokens as input:', len(hin_word_tokens))

Number of word tokens as input: 8640033


In [ ]:
print('Number of trigrams obtained:', len(hin_trigrams))
print('Sample output', hin_trigrams[0:10])

Number of trigrams obtained: 8640031
Sample output ['मास्टर ऑफ़ हेल्थ', 'ऑफ़ हेल्थ एडमिनिस्ट्रेशन', 'हेल्थ एडमिनिस्ट्रेशन या', 'एडमिनिस्ट्रेशन या मास्टर', 'या मास्टर ऑफ', 'मास्टर ऑफ हेल्थकेयर', 'ऑफ हेल्थकेयर एडमिनिस्ट्रेशन', 'हेल्थकेयर एडमिनिस्ट्रेशन (', 'एडमिनिस्ट्रेशन ( एमएचए', '( एमएचए या']


In [ ]:
hin_tri_freq_dist = nltk.FreqDist(hin_trigrams)
plot_freq_dist(hin_tri_freq_dist, 3, "Hindi")
print('Type of trigrams:', len(hin_tri_freq_dist))
print('Sample Output:', hin_tri_freq_dist.most_common(5))

Type of trigrams: 5508737
Sample Output: [('के रूप में', 16475), ('जाता है ।', 14139), ('करने के लिए', 7987), ('होता है ।', 7927), ('किया जाता है', 6457)]


### 1.3.2 - Few Basic Questions

In [59]:
def num_coverage(freq_dist, ratio):
  freq_list = np.array(list(reversed(sorted([val for _, val in freq_dist.items()]))))
  threshold_count = np.argmin(freq_list.cumsum() < freq_dist.N()*0.9)
  return threshold_count

#### 1 - 90% Coverage by unigrams
Dependencies: eng_uni_freq_dist, hin_uni_freq_dist

##### English

In [ ]:
threshold_count = num_coverage(eng_uni_freq_dist, 0.9)
print('Type of unigrams as input:', len(eng_uni_freq_dist))
print('Number of unigrams required for 90% coverage: ' + str(threshold_count))

Type of unigrams as input: 286686
Number of unigrams required for 90% coverage: 11224


##### Hindi

In [ ]:
threshold_count = num_coverage(hin_uni_freq_dist, 0.9)
print('Type of unigrams as input:', len(hin_uni_freq_dist))
print('Number of unigrams required for 90% coverage: ' + str(threshold_count))

Type of unigrams as input: 322350
Number of unigrams required for 90% coverage: 13339


#### 2 - 80% Coverage by bigrams
Dependencies: eng_bi_freq_dist, hin_bi_freq_dist

##### English

In [ ]:
threshold_count = num_coverage(eng_bi_freq_dist, 0.8)
print('Type of bigrams as input:', len(eng_bi_freq_dist))
print('Number of bigrams required for 80% coverage: ' + str(threshold_count))

Type of bigrams as input: 3977129
Number of bigrams required for 80% coverage: 1939876


##### Hindi

In [ ]:
threshold_count = num_coverage(hin_bi_freq_dist, 0.8)
print('Type of bigrams as input:', len(hin_bi_freq_dist))
print('Number of bigrams required for 80% coverage: ' + str(threshold_count))

Type of bigrams as input: 2392979
Number of bigrams required for 80% coverage: 1528975


#### 3 - 70% Coverage by trigrams
Dependencies: eng_tri_freq_dist, hin_tri_freq_dist

##### English

In [ ]:
threshold_count = num_coverage(eng_tri_freq_dist, 0.7)
print('Type of trigrams as input:', len(eng_tri_freq_dist))
print('Number of trigrams required for 70% coverage: ' + str(threshold_count))

Type of trigrams as input: 10878682
Number of trigrams required for 70% coverage: 8841429


##### Hindi

In [ ]:
threshold_count = num_coverage(hin_tri_freq_dist, 0.7)
print('Type of trigrams as input:', len(hin_tri_freq_dist))
print('Number of trigrams required for 70% coverage: ' + str(threshold_count))

Type of trigrams as input: 5508737
Number of trigrams required for 70% coverage: 4644733


#### 4 - Coverage after stemming
Dependencies: eng_word_tokens

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
eng_word_tokens_stem = [ps.stem(word) for word in eng_word_tokens]

Number of word tokens as input: 20372526
Number of stemmed word tokens as input: 20372526


In [ ]:
print('Number of word tokens as input:', len(eng_word_tokens))
print('Sample Output:', eng_word_tokens_stem[0:20])

Number of word tokens as input: 20372526
Sample Output: ['the', 'word', '"', 'atom', '"', 'wa', 'coin', 'by', 'ancient', 'greek', 'philosoph', '.', 'howev', ',', 'these', 'idea', 'were', 'found', 'in', 'philosoph']


In [72]:
def generate_hin_stem_words(word):
  suffixes = {
    1: [u"ो",u"े",u"ू",u"ु",u"ी",u"ि",u"ा"],
    2: [u"कर",u"ाओ",u"िए",u"ाई",u"ाए",u"ने",u"नी",u"ना",u"ते",u"ीं",u"ती",u"ता",u"ाँ",u"ां",u"ों",u"ें"],
    3: [u"ाकर",u"ाइए",u"ाईं",u"ाया",u"ेगी",u"ेगा",u"ोगी",u"ोगे",u"ाने",u"ाना",u"ाते",u"ाती",u"ाता",u"तीं",u"ाओं",u"ाएं",u"ुओं",u"ुएं",u"ुआं"],
    4: [u"ाएगी",u"ाएगा",u"ाओगी",u"ाओगे",u"एंगी",u"ेंगी",u"एंगे",u"ेंगे",u"ूंगी",u"ूंगा",u"ातीं",u"नाओं",u"नाएं",u"ताओं",u"ताएं",u"ियाँ",u"ियों",u"ियां"],
    5: [u"ाएंगी",u"ाएंगे",u"ाऊंगी",u"ाऊंगा",u"ाइयाँ",u"ाइयों",u"ाइयां"],
  }
  
  for L in 5, 4, 3, 2, 1:
    if len(word) > L + 1:
      for suf in suffixes[L]:
        if word.endswith(suf):
          return word[:-L]
  return word

In [74]:
print('Number of word tokens as input:', len(hin_word_tokens))
hin_word_tokens_stem = [generate_hin_stem_words(word) for word in hin_word_tokens]
print('Sample Output:', hin_word_tokens_stem[0:30])

Number of word tokens as input: 8640033
Sample Output: ['मास्टर', 'ऑफ़', 'हेल्थ', 'एडमिनिस्ट्रेशन', 'या', 'मास्टर', 'ऑफ', 'हेल्थकेयर', 'एडमिनिस्ट्रेशन', '(', 'एमएचए', 'या', 'एम', '.', 'एच', '.', 'ए', ')', 'स्नातकोत्तर', '(', 'पोस्ट', 'ग्रेजुएशन', ')', 'की', 'एक', 'पेशेवर', 'डिग्र', 'है', 'जो', 'स्वास्थ्य']


##### Unigrams - English

In [ ]:
eng_unigrams = extract_ngrams(eng_word_tokens_stem, 1)
eng_uni_freq_dist = nltk.FreqDist(eng_unigrams)
print('Type of unigrams:', len(eng_uni_freq_dist))
print('Sample Output:', eng_uni_freq_dist.most_common(10))

Type of unigrams: 200158
Sample Output: [('the', 1270134), (',', 994858), ('.', 862133), ('of', 646927), ('and', 501314), ('in', 379195), ('to', 356140), ('a', 334475), ('"', 308618), ('wa', 210603)]


In [ ]:
threshold_count = num_coverage(eng_uni_freq_dist, 0.9)
print('Type of unigrams required for 90% coverage: ' + str(threshold_count))

Number of unigrams required for 90% coverage: 4499


##### Unigrams - Hindi

In [75]:
hin_unigrams = extract_ngrams(hin_word_tokens_stem, 1)
hin_uni_freq_dist = nltk.FreqDist(hin_unigrams)
print('Type of unigrams:', len(hin_uni_freq_dist))
print('Sample Output:', hin_uni_freq_dist.most_common(10))

Type of unigrams: 279722
Sample Output: [('के', 357866), ('।', 321526), ('में', 268249), (',', 267743), ('है', 232300), ('की', 171786), ('और', 155765), ('से', 138136), ('का', 123027), ('को', 116110)]


In [76]:
threshold_count = num_coverage(hin_uni_freq_dist, 0.9)
print('Type of unigrams required for 90% coverage: ' + str(threshold_count))

Type of unigrams required for 90% coverage: 8033


##### Bigrams - English

In [ ]:
eng_bigrams = extract_ngrams(eng_word_tokens_stem, 2)
eng_bi_freq_dist = nltk.FreqDist(eng_bigrams)
print('Type of bigrams:', len(eng_bi_freq_dist))
print('Sample Output:', eng_bi_freq_dist.most_common(10))

Type of bigrams: 3184100
Sample Output: [('of the', 179997), ('. the', 140918), (', and', 114466), ("' s", 105543), ('in the', 105047), (', the', 74499), ('to the', 58391), ('. In', 56002), ('and the', 47938), ('on the', 33052)]


In [ ]:
threshold_count = num_coverage(eng_bi_freq_dist, 0.8)
print('Type of bigrams required for 80% coverage: ' + str(threshold_count))

Type of bigrams required for 80% coverage: 1146847


##### Bigrams - Hindi

In [77]:
hin_bigrams = extract_ngrams(hin_word_tokens_stem, 2)
hin_bi_freq_dist = nltk.FreqDist(hin_bigrams)
print('Type of bigrams:', len(hin_bi_freq_dist))
print('Sample Output:', hin_bi_freq_dist.most_common(10))

Type of bigrams: 2117887
Sample Output: [('है ।', 125371), ('के लिए', 49252), ('हैं ।', 48556), ('जा है', 32300), ('है ,', 32263), ('था ।', 24260), ('हो है', 23275), ('के साथ', 21080), ('रूप में', 18277), ('के रूप', 16839)]


In [78]:
threshold_count = num_coverage(hin_bi_freq_dist, 0.8)
print('Type of bigrams required for 80% coverage: ' + str(threshold_count))

Type of bigrams required for 80% coverage: 1253883


##### Trigrams - English

In [ ]:
eng_trigrams = extract_ngrams(eng_word_tokens_stem, 3)
eng_tri_freq_dist = nltk.FreqDist(eng_trigrams)

Type of bigrams: 10145300
Sample Output: [(', and the', 14485), ('. In the', 13255), ('the age of', 12013), ('age of 18', 10081), ('under the age', 9929), ('the unit state', 9773), ('year of age', 9262), ('or older .', 9118), ('older . the', 9107), ('age or older', 9092)]


In [ ]:
print('Type of bigrams:', len(eng_tri_freq_dist))
print('Sample Output:', eng_tri_freq_dist.most_common(50))

Type of bigrams: 10145300
Sample Output: [(', and the', 14485), ('. In the', 13255), ('the age of', 12013), ('age of 18', 10081), ('under the age', 9929), ('the unit state', 9773), ('year of age', 9262), ('or older .', 9118), ('older . the', 9107), ('age or older', 9092), ('of age or', 9091), ('65 year of', 9088), ('% of the', 8875), (', there were', 8832), ('one of the', 8620), ('. the median', 8322), ('of the popul', 8177), ('. 3 %', 7929), (', 0 .', 7573), ('. the popul', 7536), ('. 1 %', 7501), ('. for everi', 7458), ('for everi 100', 7453), ('everi 100 femal', 7444), ('. 5 %', 7407), ('. 2 %', 7393), ('. 8 %', 7390), ('. 4 %', 7346), ('incom for a', 7315), ('median incom for', 7309), ('the median incom', 7306), ('squar mile (', 7275), ('. 6 %', 7224), ('. 0 %', 7214), ('. 7 %', 7211), ('. 9 %', 7000), ('/ km² ).', 6938), ('U . S', 6867), ('the popul wa', 6858), ('. howev ,', 6826), ('. S .', 6802), ('as well as', 6590), ('. there were', 6521), ('. accord to', 6507), ('the citi wa'

In [ ]:
threshold_count = num_coverage(eng_tri_freq_dist, 0.7)
print('Type of trigrams required for 70% coverage: ' + str(threshold_count))

Type of trigrams required for 70% coverage: 8108047


##### Trigrams - Hindi

In [79]:
hin_trigrams = extract_ngrams(hin_word_tokens_stem, 3)
hin_tri_freq_dist = nltk.FreqDist(hin_trigrams)
print('Type of bigrams:', len(hin_tri_freq_dist))
print('Sample Output:', hin_tri_freq_dist.most_common(50))

Type of bigrams: 5242642
Sample Output: [('जा है ।', 18479), ('के रूप में', 16488), ('हो है ।', 12505), ('कर के लिए', 8091), ('है । यह', 7876), ('कर है ।', 7606), ('सक है ।', 7332), ('किय जा है', 6533), ('है । इस', 6203), ('जा सक है', 5920), ('गय है ।', 5849), ('रह है ।', 5661), ('गय था ।', 5531), ('कर हैं ।', 4804), ('हो हैं ।', 4630), ('जा है ,', 4442), ('किय गय था', 4377), ('है । इसक', 4272), ('जा हैं ।', 4119), ('है , जो', 3741), ('हो जा है', 3691), ('हो है ,', 3649), ('किय गय है', 3555), ('किय गय ।', 3374), ('हो सक है', 3326), ('सक हैं ।', 3155), ('किय जा सक', 2976), ('रह हैं ।', 2935), ('के बार में', 2862), ('कह जा है', 2793), ('के लिए एक', 2786), ('में से एक', 2533), ('मा जा है', 2439), ('दे है ।', 2420), ('जा है और', 2325), ('के माध्यम से', 2292), ('किय है ।', 2262), ('कर है ,', 2188), ('के लिए ,', 2165), ('नह है ।', 2126), ('के नाम से', 2069), ('किय था ।', 1991), ('जा जा है', 1991), ('हो गय ।', 1989), ('के आधार पर', 1958), ('हैं । यह', 1929), ('कह हैं ।', 1904), ('के बाद ,', 1

In [80]:
threshold_count = num_coverage(hin_tri_freq_dist, 0.7)
print('Type of trigrams required for 70% coverage: ' + str(threshold_count))

Type of trigrams required for 70% coverage: 4378638


### 1.3.3 - Writing some of your basic codes and comparing with results obtained using tools

#### 1 - Heuristics For Sentence Segmentation And Word Tokenization
Dependencies: eng_text, hin_text

##### English

In [ ]:
def eng_heuristics(text):
  pattern = r'''(?x)          # set flag to allow verbose regexps
  (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
  | (?:\s\w\w\.)+         # Dr. Mr. Ms.
  | \w+(?:-\w+)*        # words with optional internal hyphens
  | \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
  | \.\.\.              # ellipsis
  | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
  '''
  sentences = nltk.tokenize.RegexpTokenizer(pattern).tokenize(text)
  
  all_words = []
  for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    all_words += words
  return all_words

def ignoreCase(tokens):
    tokens = [token.lower() for token in tokens]
    return tokens

In [ ]:
eng_word_tokens = eng_heuristics(eng_text)

In [ ]:
eng_word_tokens = ignoreCase(eng_word_tokens)

In [ ]:
eng_unigrams = extract_ngrams(eng_word_tokens, 1)
eng_uni_freq_dist = nltk.FreqDist(eng_unigrams)
threshold_count = num_coverage(eng_uni_freq_dist, 0.9)
print('Number of unigrams:', len(eng_unigrams))
print('Type of unigrams as input:', len(eng_uni_freq_dist))
print('Number of unigrams required for 90% coverage: ' + str(threshold_count))

Number of unigrams: 20065097
Type of unigrams as input: 297135
Number of unigrams required for 90% coverage: 9296


In [ ]:
eng_bigrams = extract_ngrams(eng_word_tokens, 2)
eng_bi_freq_dist = nltk.FreqDist(eng_bigrams)
threshold_count = num_coverage(eng_bi_freq_dist, 0.8)
print('Number of bigrams:', len(eng_bigrams))
print('Type of bigrams as input:', len(eng_bi_freq_dist))
print('Number of bigrams required for 80% coverage: ' + str(threshold_count))

Number of bigrams: 20065096
Type of bigrams as input: 3687791
Number of bigrams required for 80% coverage: 1681281


In [ ]:
eng_trigrams = extract_ngrams(eng_word_tokens, 3)
eng_tri_freq_dist = nltk.FreqDist(eng_trigrams)
threshold_count = num_coverage(eng_tri_freq_dist, 0.7)
print('Number of trigrams:', len(eng_trigrams))
print('Type of trigrams as input:', len(eng_tri_freq_dist))
print('Number of trigrams required for 70% coverage: ' + str(threshold_count))

Number of trigrams: 20065095
Type of trigrams as input: 10392479
Number of trigrams required for 70% coverage: 8385969


##### Hindi

In [54]:
def hin_heuristics(text):
  sentences = text.split(u"।")
  
  all_words = []
  for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    all_words += words
  return all_words

In [57]:
hin_word_tokens = hin_heuristics(hin_text)

In [60]:
hin_unigrams = extract_ngrams(hin_word_tokens, 1)
hin_uni_freq_dist = nltk.FreqDist(hin_unigrams)
threshold_count = num_coverage(hin_uni_freq_dist, 0.9)
print('Number of unigrams:', len(hin_unigrams))
print('Type of unigrams as input:', len(hin_uni_freq_dist))
print('Number of unigrams required for 90% coverage: ' + str(threshold_count))

Number of unigrams: 8158694
Type of unigrams as input: 317747
Number of unigrams required for 90% coverage: 13860


In [61]:
hin_bigrams = extract_ngrams(hin_word_tokens, 2)
hin_bi_freq_dist = nltk.FreqDist(hin_bigrams)
threshold_count = num_coverage(hin_bi_freq_dist, 0.8)
print('Number of bigrams:', len(hin_bigrams))
print('Type of bigrams as input:', len(hin_bi_freq_dist))
print('Number of bigrams required for 80% coverage: ' + str(threshold_count))

Number of bigrams: 8158693
Type of bigrams as input: 2407639
Number of bigrams required for 80% coverage: 1591769


In [62]:
hin_trigrams = extract_ngrams(hin_word_tokens, 3)
hin_tri_freq_dist = nltk.FreqDist(hin_trigrams)
threshold_count = num_coverage(hin_tri_freq_dist, 0.7)
print('Number of trigrams:', len(hin_trigrams))
print('Type of trigrams as input:', len(hin_tri_freq_dist))
print('Number of trigrams required for 70% coverage: ' + str(threshold_count))

Number of trigrams: 8158692
Type of trigrams as input: 5520700
Number of trigrams required for 70% coverage: 4704830


#### 2 - Likelihood Ratio Test
Dependencies: eng_bi_freq_dist, eng_uni_freq_dist, eng_unigrams, hin_bi_freq_dist, hin_uni_freq_dist, hin_unigrams 

In [ ]:
from math import log10

def getVal(k, n, x):
  temp = log10(x) * k
  temp2 = log10(1-x) * (n-k)
  temp += temp2
  return temp

def constructCollocations(bigram_dist, unigram_dist, number_tokens):
    collocation = []
    i = 0
    for bigram, freq in bigram_dist.items():
      bigramSplit = bigram.split()
      c12 = freq
      if len(bigramSplit) < 2:
        continue

      c1 = unigram_dist[bigramSplit[0]]
      c2 = unigram_dist[bigramSplit[1]]
      n = number_tokens
      if c1 == 0 or c1 == n:
        continue
      
      p = c2/n
      p1 = c12/c1
      p2 = (c2 - c12)/(n-c1)
      if(p2 <= 0 or p1 <= 0 or p <= 0):
          continue

      if(p2 >= 1 or p1 >= 1 or p >= 1):
          continue

      val = getVal(c12, c1, p) + getVal(c2 - c12, n-c1, p) - getVal(c12, c1, p1) - getVal(c2 - c12, n-c1, p2)
      val *= -2

      if(val >= 7.88):
          collocation.append(bigram)

    return collocation

##### English

In [ ]:
print('Type of bigrams as input:', len(eng_bi_freq_dist))
print('Type of unigrams as input:', len(eng_uni_freq_dist))
print('Number of unigrams as input:', len(eng_unigrams))
collocations = constructCollocations(eng_bi_freq_dist, eng_uni_freq_dist, len(eng_unigrams))
print('Number of collocations obtained:', len(collocations))
print('Sample Ouput:', collocations[0:20])

Type of bigrams as input: 3977129
Type of unigrams as input: 286686
Number of unigrams as input: 20372526
Number of collocations obtained: 491686
Sample Ouput: ['The word', 'word "', '" was', 'was coined', 'coined by', 'ancient Greek', 'Greek philosophers', '. However', 'However ,', ', these', 'these ideas', 'ideas were', 'were founded', 'founded in', 'philosophical and', 'and theological', 'rather than', 'and experimentation', '. As', 'As a']


##### Hindi

In [ ]:
print('Type of bigrams as input:', len(hin_bi_freq_dist))
print('Type of unigrams as input:', len(hin_uni_freq_dist))
print('Number of unigrams as input:', len(hin_unigrams))
collocations = constructCollocations(hin_bi_freq_dist, hin_uni_freq_dist, len(hin_unigrams))
print('Number of collocations obtained:', len(collocations))
print('Sample Ouput:', collocations[0:20])

Type of bigrams as input: 2392979
Type of unigrams as input: 322350
Number of unigrams as input: 8640033
Number of collocations obtained: 262625
Sample Ouput: ['मास्टर ऑफ़', 'ऑफ़ हेल्थ', 'हेल्थ एडमिनिस्ट्रेशन', 'मास्टर ऑफ', 'हेल्थकेयर एडमिनिस्ट्रेशन', 'एडमिनिस्ट्रेशन (', 'एम .', '. एच', 'एच .', '. ए', 'स्नातकोत्तर (', 'पोस्ट ग्रेजुएशन', ') की', 'की एक', 'एक पेशेवर', 'है जो', 'स्वास्थ्य प्रशासन', 'प्रशासन के', 'के क्षेत्र', 'क्षेत्र में']


### 1.3.4 - Morphological parsing

#### 1 - Morphological Analysis of most-frequent and least-frequent words

In [ ]:
import random

In [ ]:
def random_freq_unigrams(freq_dist, n, m, isLeast):
  random_unigrams = []
  if isLeast:
    freq_unigrams = freq_dist.most_common()[-n:]
    random_unigrams = random.choices(freq_unigrams, k=m)
  else:
    freq_unigrams = freq_dist.most_common(n)
    random_unigrams = random.choices(freq_unigrams, k=m)
  return random_unigrams

##### Morphological Analysis - English
Dependencies: eng_uni_freq_dist

In [ ]:
!sudo apt-get install python-numpy libicu-dev
!pip install PyICU polyglot pycld2 Morfessor

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-numpy is already the newest version (1:1.13.3-2ubuntu1).
python-numpy set to manually installed.
libicu-dev is already the newest version (60.2-3ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
     |████████████████████████████████| 235kB 7.3MB/s 
     |████████████████████████████████| 133kB 13.6MB/s 
     |████████████████████████████████| 41.4MB 94kB/s 
  Created wheel for PyICU: filename=PyICU-2.5-cp36-cp36m-linux_x86_64.whl size=1252650 sha256=42e23492880999eedea4956ab256ae9c974f64ff21001b342f9cb6fef4ca8a95
  Stored in directory: /root/.cache/pip/wheels/07/3a/28/09f90c38785945ddf9af61b7add1aa62a740f40e259626ef3a
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52559 sha256=bddc98ec935a789222f564f044721272cd96eafae93f8e758ad72b0596d505ad
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b

In [ ]:
from polyglot.downloader import downloader

In [ ]:
!polyglot download morph2.en

[polyglot_data] Downloading package morph2.en to
[polyglot_data]     /root/polyglot_data...


In [ ]:
from polyglot.text import Word

In [ ]:
print('Type of words as input:', len(eng_uni_freq_dist))
random_unigrams = random_freq_unigrams(eng_uni_freq_dist, 100, 5, False)
random_freq_words = []
for word, freq in random_unigrams:
  random_freq_words.append(word)
print('Random words chosen:', random_freq_words)

Type of words as input: 286686
Random words chosen: ['can', 'than', 'their', 'or', 'been']


In [ ]:
for word in random_freq_words:
  word = Word(word, language="en")
  print("{:<20}{}".format(word, word.morphemes))

can                 ['can']
than                ['th', 'an']
their               ['t', 'heir']
or                  ['or']
been                ['be', 'en']


In [ ]:
random_unigrams = random_freq_unigrams(eng_uni_freq_dist, 100, 5, True)
random_words = []
for word, freq in random_unigrams:
  random_words.append(word)
print('Random words chosen:', random_words)

Random words chosen: ['Karun', 'ellipticities', 'Thalaimurai', 'DECsystem', 'Gorillas']


In [ ]:
for word in random_words:
  word = Word(word, language="en")
  print("{:<20}{}".format(word, word.morphemes))

Karun               ['Ka', 'run']
ellipticities       ['elliptic', 'ities']
Thalaimurai         ['Th', 'al', 'aim', 'urai']
DECsystem           ['DE', 'C', 'system']
Gorillas            ['Gor', 'illa', 's']


##### Morphological Analysis - Hindi
Dependencies: hin_uni_freq_dist

In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 24.11 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Checking out files: 100% (28/28), done.


In [ ]:
from indicnlp.morph import unsupervised_morph 
from indicnlp import common
common.INDIC_RESOURCES_PATH="/content/indic_nlp_resources"
analyzer = unsupervised_morph.UnsupervisedMorphAnalyzer('hi')

In [ ]:
print('Type of unigrams as input:', len(hin_uni_freq_dist))
random_unigrams = random_freq_unigrams(hin_uni_freq_dist, 100, 5, False)
random_freq_words = []
for word, freq in random_unigrams:
  random_freq_words.append(word)
print(random_freq_words)

Type of unigrams as input: 322350
['ही', 'हुई', 'क्षेत्र', 'थे', 'गए']


In [ ]:
for word in random_freq_words:
  hin_morph = analyzer.morph_analyze(word)
  print("{:<20}{}".format(word, hin_morph))

ही                  ['ही']
हुई                 ['हुई']
क्षेत्र             ['क्षेत्र']
थे                  ['थे']
गए                  ['गए']


In [ ]:
random_unigrams = random_freq_unigrams(hin_uni_freq_dist, 100, 5, True)
random_words = []
for word, freq in random_unigrams:
  random_words.append(word)
print(random_words)

['उहेल्स', 'केंद्रकसूत्रों', 'थेलिओटोकी', 'स्त्रीजनन', 'फ़्ली']


In [ ]:
for word in random_words:
  hin_morph = analyzer.morph_analyze(word)
  print("{:<20}{}".format(word, hin_morph))

उहेल्स              ['उ', 'हेल्स']
केंद्रकसूत्रों      ['केंद्र', 'क', 'सूत्रों']
थेलिओटोकी           ['थे', 'लिओ', 'टोकी']
स्त्रीजनन           ['स्त्री', 'जन', 'न']
फ़्ली               ['फ़्', 'ली']


### 1.3.5 - Sub-word tokenization
Dependencies: eng_uni_freq_dist, hin_uni_freq_dist

In [ ]:
import re
from collections import defaultdict

In [ ]:
def build_vocab(freq_dist):
  vocab = nltk.FreqDist([])
  for word, freq in freq_dist.items():
    w = ''
    for c in word:
      w += c + ' '
    vocab[w+'</w>'] = freq
  return vocab

def get_stats(vocab):
    pairs = defaultdict(int)
    for word, frequency in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i + 1]] += frequency
    return pairs


def merge_vocab(pair, v_in):
    v_out =  nltk.FreqDist([])
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def byte_pair_encoding(freq_dist, num):
  vocab = build_vocab(freq_dist)
  num_merges = num
  print('Vocabulary size:', num_merges)
  fin_encodings = []

  for i in range(num_merges):
      pairs = get_stats(vocab)

      if not pairs:
          break

      best = max(pairs, key=pairs.get)
      fin_encodings.append(best)
      vocab = merge_vocab(best, vocab)

  return vocab, fin_encodings

def merge_vocab_list(pair, v_in):
    v_out = []
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
   
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out.append(w_out)

    return v_out

def tokenizeCorpus(corpus, finPairs):
    words = [" ".join(word) + " </w>" for word in corpus.split()]
    for pair in finPairs:
      words = merge_vocab_list(pair, words)
    return words

##### English

In [ ]:
print('Type of words as input:', len(eng_uni_freq_dist))

Type of words as input: 58731


In [ ]:
fin_encodings = byte_pair_encoding(eng_uni_freq_dist, 4000)
print('Final byte encodings:', fin_encodings[0:20])

Vocabulary size: 4000
Final byte encodings: [('e', '</w>'), ('s', '</w>'), ('t', 'h'), ('e', 'r'), ('.', '</w>'), ('d', '</w>'), ('n', '</w>'), (',', '</w>'), ('th', 'e</w>'), ('i', 'n'), ('a', 'n'), ('o', 'f'), ('of', '</w>'), ('y', '</w>'), ('t', '</w>'), ('o', 'r'), ('a', 'l'), ('a', 'r'), ('an', 'd</w>'), ('er', '</w>')]


In [ ]:
print('Most frequent words:', vocab.most_common(50))

Most frequent words: [('.</w>', 153214), (',</w>', 116642), ('the</w>', 102691), ('of</w>', 87515), ('and</w>', 54311), ('%</w>', 48617), ('was</w>', 36350), ('in</w>', 32048), ('a</w>', 31982), ('were</w>', 30573), ('to</w>', 28767), ('The</w>', 27373), ('is</w>', 17501), ('age</w>', 17249), ('"</w>', 16173), ('from</w>', 15099), ('for</w>', 14914), ('or</w>', 14196), ('with</w>', 13567), ('(</w>', 13466), ('0</w>', 12440), ('2</w>', 12431), ('-</w>', 12147), ('had</w>', 12120), ('18</w>', 12067), ('1</w>', 11262), ('population</w>', 10763), ('township</w>', 10393), ('3</w>', 10224), ('$</w>', 10012), ('as</w>', 8989), ('years</w>', 8496), ('city</w>', 8414), ('median</w>', 8354), ('living</w>', 8141), ('4</w>', 8053), ('5</w>', 7896), ('which</w>', 7813), ('income</w>', 7806), ('average</w>', 7737), ('at</w>', 7661), ('households</w>', 7600), ('6</w>', 7544), ('8</w>', 7481), ('under</w>', 7459), ('7</w>', 7377), ('by</w>', 7335), ('there</w>', 7296), ('families</w>', 7079), ("'</w>"

In [ ]:
print('Least frequent words:', vocab.most_common()[-50:])

Least frequent words: [('K ra use</w>', 1), ('Al bers</w>', 1), ('Fre der ick son</w>', 1), ('Mus cal a</w>', 1), ('A top</w>', 1), ('L in k ar a</w>', 1), ('Lov ha u g</w>', 1), ('G on e au</w>', 1), ('L or en</w>', 1), ('Bri m h all</w>', 1), ('Em met t</w>', 1), ('Par k view</w>', 1), ('R A M S</w>', 1), ('S n ail</w>', 1), ('O w as so</w>', 1), ('F em ales</w>', 1), ('An ce str ies</w>', 1), ('O d il ia</w>', 1), ('polic y making</w>', 1), ('Mar i ann e</w>', 1), ('bro ad c ast ers</w>', 1), ('ant enn ae</w>', 1), ('T ele f arm</w>', 1), ('Re plac ement</w>', 1), ('lat er als</w>', 1), ('S ew er</w>', 1), ('W ast ew ater</w>', 1), ('M C W S</w>', 1), ('P uc k</w>', 1), ('Mc M ill an</w>', 1), ('in a de qu acy</w>', 1), ('out building</w>', 1), ('C er t ification</w>', 1), ('tr ic ol our</w>', 1), ('dis cont igu ous</w>', 1), ('Ter re b on ne</w>', 1), ('Par ent e au</w>', 1), ('bu t ter f at</w>', 1), ('sk im</w>', 1), ('wh ey</w>', 1), ('mon oc ul ture</w>', 1), ('pre domin ated</

In [ ]:
corpus = 'serendipity gobbledygook scrumptious agastopia halfpace impignorate jentacular nudiustertian quire yarborough'
morph_list = tokenizeCorpus(corpus, fin_encodings)
print(morph_list)

['ser en di p ity</w>', 'g ob b le d y g ook</w>', 'sc r um p t ious</w>', 'ag as top ia</w>', 'hal f p ace</w>', 'imp ig nor ate</w>', 'j ent ac ular</w>', 'nu di u ster t ian</w>', 'qu ire</w>', 'y ar bor ough</w>']


In [ ]:
corpus = corpus.split()
for word in corpus:
  word = Word(word, language="en")
  print("{:<20}{}".format(word, word.morphemes))

serendipity         ['s', 'er', 'en', 'dip', 'ity']
gobbledygook        ['go', 'b', 'ble', 'dy', 'go', 'o', 'k']
scrumptious         ['s', 'c', 'rump', 't', 'ious']
agastopia           ['a', 'ga', 'stop', 'ia']
halfpace            ['half', 'pace']
impignorate         ['imp', 'ignor', 'ate']
jentacular          ['je', 'nta', 'cular']
nudiustertian       ['nudi', 'us', 'ter', 'tian']
quire               ['quire']
yarborough          ['y', 'ar', 'borough']


##### Hindi

In [ ]:
print('Type of words as input:', len(hin_uni_freq_dist))

Type of words as input: 63290


In [ ]:
vocab, fin_encodings = byte_pair_encoding(hin_uni_freq_dist, 4000)
print('Final byte encodings:', fin_encodings[0:20])

Vocabulary size: 4000
Final byte encodings: [('े', '</w>'), ('ा', '</w>'), ('ी', '</w>'), ('ं', '</w>'), ('र', '</w>'), ('क', 'े</w>'), ('्', 'र'), ('न', '</w>'), ('ह', 'ै'), ('।', '</w>'), ('म', 'े'), ('मे', 'ं</w>'), ('क', '</w>'), (',', '</w>'), ('र', '्'), ('त', '</w>'), ('ो', '</w>'), ('है', '</w>'), ('्', 'य'), ('ि', 'य')]


In [ ]:
print('Most frequent words:', vocab.most_common(50))

Most frequent words: [('के</w>', 34914), ('।</w>', 31925), (',</w>', 27641), ('में</w>', 26777), ('है</w>', 23026), ('की</w>', 16905), ('और</w>', 15511), ('से</w>', 13455), ('का</w>', 12385), ('को</w>', 11851), ('"</w>', 9234), ('हैं</w>', 8764), ('एक</w>', 8006), ('-</w>', 7802), (')</w>', 6683), ('(</w>', 6653), ('पर</w>', 6371), ('ने</w>', 5452), ('किया</w>', 5448), ('लिए</w>', 5008), ('भी</w>', 4939), ('.</w>', 4408), ('था</w>', 4189), ('कि</w>', 4175), ('गया</w>', 3824), ('यह</w>', 3758), ('इस</w>', 3501), ('रूप</w>', 3416), ('जो</w>', 3002), ('ही</w>', 2990), ('जाता</w>', 2957), ('नहीं</w>', 2836), ('करने</w>', 2827), ('साथ</w>', 2788), ('कर</w>', 2745), ('हो</w>', 2742), ('या</w>', 2471), ('द्वारा</w>', 2356), ('थे</w>', 2236), ('तथा</w>', 2136), ("'</w>", 2131), ('अपने</w>', 1990), ('दिया</w>', 1877), ('होता</w>', 1868), (':</w>', 1805), ('तक</w>', 1746), ('बाद</w>', 1730), ('थी</w>', 1668), ('वह</w>', 1660), ('करते</w>', 1503)]


In [ ]:
print('Least frequent words:', vocab.most_common()[-50:])

Least frequent words: [('अ सर दार</w>', 1), ('“ प ण्ड ित</w>', 1), ('जग दी श च न्द्र</w>', 1), ('सम् मो हित</w>', 1), ('\n ज श पुर</w>', 1), ('\n \n छ त् तीस गढ़</w>', 1), ('नी चा घा ट</w>', 1), ('रो मं चक</w>', 1), ('उ रा ओं</w>', 1), ('ध ं धा</w>', 1), ('को टे बि रा</w>', 1), ('ए ब</w>', 1), ('\n \n सम र बार</w>', 1), ('बु झ ा</w>', 1), ('महा गि रि जा घर</w>', 1), ('नि हा र ने</w>', 1), ('कैम र</w>', 1), ('द नग री</w>', 1), ('टै क् सियों</w>', 1), ('\n पुष् प कवि मान</w>', 1), ('सु वर्ण म ण्ड ित</w>', 1), ('प्रार ु प</w>', 1), ('अं गि रा</w>', 1), ('भि ड ंत</w>', 1), ('उ ड ान</w>', 1), ('भर ते</w>', 1), ('उ डन</w>', 1), ('त श तर ियों</w>', 1), ('न भ चर</w>', 1), ('स्वय मे व</w>', 1), ('अल का पुरी</w>', 1), ('\n \n वर् त् त मान</w>', 1), ('वि मान क्षेत्रों</w>', 1), ('उ सा न गो ड़ा</w>', 1), ('गु रू लो पो था</w>', 1), ('तो तू पो ला क ंदा</w>', 1), ('वा रिया पो ला</w>', 1), ('\n \n ऋ ग वेद</w>', 1), ('कु मा रों</w>', 1), ('उदाह रण तया</w>', 1), ('कि ंग फि श र</w>', 1), ('जल यान</w>', 1

In [ ]:
corpus = 'आवेग अभियांत्रिकी वैतरणी असहिष्णुता कुलाधिपति चिकित्सक स्थानांतरण दस्ता संपादन प्रारूप'
morph_list = tokenizeCorpus(corpus, fin_encodings)
print(morph_list)

['आ वे ग</w>', 'अभ िया ं त्रि की</w>', 'वै तर णी</w>', 'अस हि ष्ण ु ता</w>', 'कु ला धि पति</w>', 'चिकित् स क</w>', 'स्थाना ंतरण</w>', 'द स्ता</w>', 'सं पादन</w>', 'प्रार ू प</w>']


In [ ]:
for word in corpus.split():
  hin_morph = analyzer.morph_analyze(word)
  print("{:<20}{}".format(word, hin_morph))

आवेग                ['आवेग']
अभियांत्रिकी        ['अभि', 'यांत्रिक', 'ी']
वैतरणी              ['वै', 'तरणी']
असहिष्णुता          ['अ', 'सहिष्णु', 'ता']
कुलाधिपति           ['कुला', 'धिपति']
चिकित्सक            ['चिकित्सक']
स्थानांतरण          ['स्थानांतर', 'ण']
दस्ता               ['दस्ता']
संपादन              ['संपादन']
प्रारूप             ['प्रारूप']
